In [35]:
pip install atproto

In [36]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

# import colab secrets to store login credentials
from google.colab import userdata

# datetime is necessary for caturday check
import datetime
import zoneinfo

In [37]:
# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [38]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  print('    passed cat test', would_pass)
  return would_pass

In [43]:
# on colab you can set these userdata properties by clicking the key on the left bar, creating a secret, and giving the colab "notebook access"
BSKY_USERNAME = userdata.get('bsky_username')
BSKY_PASSWORD = userdata.get('bsky_password')

client = Client()
client.login(BSKY_USERNAME, BSKY_PASSWORD)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional software developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2100%, ex streamer, ex melee player. \nGrilling, coding, magic, and posting pictures of my cat. \nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=565, follows_count=1374, indexed_at='2025-01-19T01:42:29.013Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv

In [44]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ Successfully liked post and followed author.')
  # TODO - add this friend and the followed-user uri to the database

In [97]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'
FEED_CATURDAY = 'at://did:plc:pmyqirafcp3jqdhrl7crpq7t/app.bsky.feed.generator/aaad4sb7tyvjw'
FEED_SIAMESE = 'at://did:plc:jv3qdc5vxujp6taaa7nte35i/app.bsky.feed.generator/aaac6wmikqyhq'
FEED_CATPICS = 'at://did:plc:q6gjnaw2blty4crticxkmujt/app.bsky.feed.generator/cv:cat'
FEED_CATS = 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats'
FEED_NAME = {FEED_CATURDAY: "'Caturday'", FEED_SIAMESE: "'Siamese Cats'", FEED_CATPICS: "'Cat Pics'", FEED_CATS: "'Cats!'"}
URL_BEGIN = 'https://bsky.app/profile/'
URL_POST = '/post/'

def createPostUrl(feed_post):
  url_handle = feed_post.post.author.handle
  url_ending_index = feed_post.post.uri.find('.feed.post/') + 11
  url_ending = feed_post.post.uri[url_ending_index : ]
  return URL_BEGIN + url_handle + URL_POST + url_ending

# tyrowo.bsky.social/post/3lfzygdtqis2y

def follow_more_users(post_count, follows_count, feed):
  posts_to_check = post_count
  next_page = ''
  new_follow_count = 0
  page_count = 0
  while posts_to_check > 0:
    print(f'[checking page {page_count} of feed {FEED_NAME[feed]}, {posts_to_check} posts left to check, and have found {new_follow_count} new users to follow]')
    page_count += 1
    limit = min(posts_to_check, 100)
    posts_to_check -= limit
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
        'limit': limit,
        'cursor': next_page
    }, headers={})
    next_page = data.cursor

    for i, f in enumerate(data.feed):
      you_follow_them = f.post.author.viewer.following
      you_are_followed_by = f.post.author.viewer.followed_by
      if not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
        print(f'{i} ✗ no pic for post {i}')
      elif you_follow_them or you_are_followed_by:
        print(f'{i} ✗ {f.post.author.handle + " follows you" if you_are_followed_by else ""}{" and " if you_follow_them and you_are_followed_by else ""}{"you already follow " + f.post.author.handle if you_follow_them else ""}.')
      ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
      else:
        print(i, '✓', f.post.embed.images[0].fullsize)
        print(f'    post: {createPostUrl(f)}')
        is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
        if is_cat:
          print(f'    ✓✓ successfully found cat pic at post {i}.')
          like_post_and_add_user(f.post)
          new_follow_count += 1
          if new_follow_count == follows_count:
            print(f'Successfully followed {new_follow_count} new users!')
            return True
        else:
          print(f'    ✓✗ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {new_follow_count} new users.')
  return False

    # print(data.feed[0].post.embed.images[0].fullsize)

In [96]:
CATURDAY_DOW = 'Saturday'
USER_TIMEZONE = "US/Eastern" # you should fill this in with your own timezone here
cur_timestamp = datetime.datetime.now(zoneinfo.ZoneInfo(USER_TIMEZONE))

dow = cur_timestamp.strftime("%A")
is_caturday = dow == CATURDAY_DOW

if is_caturday:
  print("IT'S CATURDAY! Checking the Caturday feed for new followers.")
  follow_more_users(1000, 100, FEED_CATURDAY)
else:
  print("Sorry, today's not Caturday.")

IT'S CATURDAY! Checking the Caturday feed for new followers.
[checking page 0 of feed 'Caturday', 1000 posts left to check]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ivbjfti62yxg72ric6srvxrg/bafkreihrssl2sllwz6w5bd6l4veua5o74nvai4bs4lw6michlviy3vjrl4@jpeg
    post: https://bsky.app/profile/eclaire.bsky.social/post/3lg2s5gwtu22t
    found cat label: 0
    found bad label: 14 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    passed cat test False
    ✓✗ post 0 was not a cat pic
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:qkm2e4dh5ucixc7pksfgkd4b/bafkreibry3w7vm3qy2xv4gbldoic5xjwcdxq4s4vgv6xgw3ixj2xoc4bzi@jpeg
    post: https://bsky.app/profile/hollidae.bsky.social/post/3lg2s57ntsc2x
    found cat label: -1
    found bad label: 48 [917, 'comic book']
    passed cat test False
    ✓✗ post 1 was not a cat pic
2 ✗ you already follow mercury811.bsky.social.
3 ✗ you already follow bolobabie.bsky.social.
4 ✓ https://cdn.bsky.app/img/f

In [98]:
follow_more_users(1000, 100, FEED_SIAMESE)

[checking page 0 of feed 'Siamese Cats', 1000 posts left to check, and have found 0 new users to follow]
0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:b5fblayo2lh4g6ife7ihcy6l/bafkreie2yei4oyvijwhu6rzkvfuqfue2lsicjwsadgxz6urp4rci56im6y@jpeg
    post: https://bsky.app/profile/owlcatjewelry.bsky.social/post/3lg2spqfd3e27


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTFeatureExtractor.preprocess` and were ignored: 'return_tensor'
  return self.preprocess(images, **kwargs)


    found cat label: 2
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 0.
      ✓✓✓ Successfully liked post and followed author.
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:6iitcjbnsnu3wpajfwgim27q/bafkreigu62xonlxzpgoy2jv3mpkg7naxybgxe24znqzxawsz2d2kwn2ewm@jpeg
    post: https://bsky.app/profile/hmlb.bsky.social/post/3lg2soz3uc22f
    found cat label: 0
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 1.
      ✓✓✓ Successfully liked post and followed author.
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:gpso2ytqicahxduss4s77oix/bafkreie6jlpstliy7grtikahuqilbjtr7ec2tdpkff5lf6zoh3p6gfgiry@jpeg
    post: https://bsky.app/profile/lolachaton822.bsky.social/post/3lg2snxmk2s2t
    found cat label: 0
    found bad label: -1 []
    passed cat test True
    ✓✓ successfully found cat pic at post 2.
      ✓✓✓ Successfully liked post and followed author.
3 ✓ https://cdn.bsky.app/img/f

True

In [49]:
follow_more_users(100, 100, FEED_CATS)

0 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:uzrm35oeukszmbg22xnfx4w3/bafkreid6zpvuuts5qxyw3x3ewm6argu6geeihafknxvwgg5ruj6v6v3cqu@jpeg
    post: at://did:plc:uzrm35oeukszmbg22xnfx4w3/app.bsky.feed.post/3lg2pbz67l22r
    found cat label: -1
    found bad label: -1 []
    passed cat test False
    ✓✗ post 0 was not a cat pic
1 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:q32j5o56ophhmuvtwf74qhlm/bafkreigalcv2o7nfpxyrg7kyu4bifi5lqkacst77achs6kfvzlos4k3bcy@jpeg
    post: at://did:plc:q32j5o56ophhmuvtwf74qhlm/app.bsky.feed.post/3lg2pbsazh22i
    found cat label: -1
    found bad label: 0 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    passed cat test False
    ✓✗ post 1 was not a cat pic
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:ivmn7ip7kjq7zawoyykr7pcx/bafkreibwzt4v6rtn3sxggb577ygrg6kykb73qqjbnowadnv3mo5y6wspla@jpeg
    post: at://did:plc:ivmn7ip7kjq7zawoyykr7pcx/app.bsky.feed.post/3lg2pbl27pc2m
    found cat label: 0


False

In [50]:
follow_more_users(100, 100, FEED_CATPICS)

0 ✗ you already follow bodegacats.bsky.social.
1 ✗ you already follow catsofyore.bsky.social.
2 ✗ you already follow radiofreetom.bsky.social.
3 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:vmhrtckqi7ghdoqhf3lgvm4y/bafkreiaijx3n2v7qujtgv22eh2ax7wc3nfmczolwoybyqujxysk7b4e4ia@jpeg
    post: at://did:plc:vmhrtckqi7ghdoqhf3lgvm4y/app.bsky.feed.post/3lg23sbd7os2u
    found cat label: 0
    found bad label: 34 [916, 'web site, website, internet site, site']
    passed cat test False
    ✓✗ post 3 was not a cat pic
4 ✗ you already follow bebeneuwirth.bsky.social.
5 ✗ you already follow abbyhiggs.bsky.social.
6 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:eohxpvnt7zuqg4cfo5if7c4s/bafkreid7acbraoz7surtlucgmbcydqrcq7pcmuewq5bn2axgpbtoy4asf4@jpeg
    post: at://did:plc:eohxpvnt7zuqg4cfo5if7c4s/app.bsky.feed.post/3lg2d3hxou22z
    found cat label: -1
    found bad label: 4 [917, 'comic book', 921, 'book jacket, dust cover, dust jacket, dust wrapper']
    passed cat test False
 

False

In [83]:
# figuring out how to recreate url of post for clickable print statement
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 30,
    'cursor': ''
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:mlhgflnobiswzziouwb6dtqk', handle='katyaandyourdad.bsky.social', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=None, labeler=None, lists=None, starter_packs=None, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:mlhgflnobiswzziouwb6dtqk/bafkreifywkwk6i7nwzd5pt4f74yzyipfxighantojsukas7dlnromv5exq@jpeg', created_at='2023-05-26T20:13:21.250Z', display_name='Jen Zamolodchikova', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreigyvrw4nlxcp6mryisyuu26uch3wthd5hz4vsizs5iheeclr5mdji', indexed_at='2025-01-19T02:22:28.215Z', record=Record(created_

In [84]:
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 1,
    'cursor': next_page
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:35osjqia5io5jftduud4xtw6', handle='kristinmathis.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:35osjqia5io5jftduud4xtw6/bafkreiebmfd7snqy2kxcsl7fbzk6t5f2gkabmiioaoyzjbemvjsorck7ay@jpeg', created_at='2024-11-14T14:27:19.856Z', display_name='Kristin Mathis', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreiabfaaxlragfne3ms6msx6ydqcnmy4jlmxlv53zaniitl2bhktdey', indexed_at='2025-01-19T02:18:36.224Z', record=Record(created_at='2025-01-19T02:18:34.248Z', text='Before #caturday is over, please enjoy Venus’ cute T-Rex paws! 🦖', embed=Main(images=[Image(alt='A black cat lies on its back with its head tilted to the left side and its paws up in the air in T-Rex p

In [85]:
next_page = ''
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 1,
    'cursor': next_page
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[] cursor='1737253653909::bafyreifihltj7f7mboofetcbai4eba7gbnmfgj766lncilpytgunoyjcgu'


In [86]:
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 1,
    'cursor': next_page
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:bjv7cqyqkygfdn4yitkwur2d', handle='kryss29.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:bjv7cqyqkygfdn4yitkwur2d/bafkreigi35gnnykqsleagaowmkermksennxciqpmuysskbrpenixq3ecey@jpeg', created_at='2024-11-13T01:21:06.241Z', display_name='Kryss29', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreibqezhs75pi2gtxm644vuzthjvqvdgoshjpohk3dlq6gf54ihhom4', indexed_at='2025-01-19T02:27:23.024Z', record=Record(created_at='2025-01-19T02:27:20.950Z', text='Hubby is putting another bookshelf together for me, and Sprinkle thought he needed some encouragement', embed=Main(images=[Image(alt='A tabby cat sits in a partially finished bookshelf and is giving her human dad a

In [87]:
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 1,
    'cursor': next_page
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:x5uyvnk4pjykfr7nyo6cpkdr', handle='falllover.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:x5uyvnk4pjykfr7nyo6cpkdr/bafkreiegk5xnft7emqtzqxgpirjsmejkwibhcdnwar3iykkbewdko5ius4@jpeg', created_at='2023-08-13T13:08:25.302Z', display_name='FallLover', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreicmuiv5mpcs6wuxuxzqspzcrpcigvrrgmsmvqxacniznnl4e3g44i', indexed_at='2025-01-19T02:27:07.623Z', record=Record(created_at='2025-01-19T02:27:03.549Z', text='The preview for #ennead Season 2, Episode 111 is up on twitter! \nx.com/witchcomics/...', embed=Main(images=[Image(alt='Seth, Horus, the human, and the cat in the preview for "ENNEAD" Season 2, Episode 112 by M

In [88]:
# figuring out how to recreate url of post for clickable print statement
data = client.app.bsky.feed.get_feed({
    'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
    'limit': 5,
    'cursor': ''
}, headers={})

feed = data.feed
next_page = data.cursor
print(data)

feed=[FeedViewPost(post=PostView(author=ProfileViewBasic(did='did:plc:bjv7cqyqkygfdn4yitkwur2d', handle='kryss29.bsky.social', associated=None, avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:bjv7cqyqkygfdn4yitkwur2d/bafkreigi35gnnykqsleagaowmkermksennxciqpmuysskbrpenixq3ecey@jpeg', created_at='2024-11-13T01:21:06.241Z', display_name='Kryss29', labels=[], viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewBasic'), cid='bafyreibqezhs75pi2gtxm644vuzthjvqvdgoshjpohk3dlq6gf54ihhom4', indexed_at='2025-01-19T02:27:23.024Z', record=Record(created_at='2025-01-19T02:27:20.950Z', text='Hubby is putting another bookshelf together for me, and Sprinkle thought he needed some encouragement', embed=Main(images=[Image(alt='A tabby cat sits in a partially finished bookshelf and is giving her human dad a